# Titre du projet ( a décider)

In [ ]:
## Sommaire

## Introduction

In [ ]:
# Install dependencies

!pip install pandas matplotlib ipywidgets kaggle

In [ ]:
# Loading data
import os
import pandas as pd

file_path = "Datasets/Data/backloggd_games.csv"

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(df.shape) # row and columns
    print(df.columns) # name/index
    df.head() # Preview

else:
    print("The file does not exist. Check the path or place the file in the correct location")
